The overall idea is that we will implement linear and tree based models first and try to ansamble them. Further will implement recurrent NN and compare the two models.
Data normalisation required for linear or NN models will be done with the help of sklearn pipeline which will include both motel and data transofrmation steps. 

In [11]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from src.ToyModel import *
from src.TestGenerator import *
from src.utilities import run_cv
from src.FeatureGenerator import *
from src.FeatureGenerator import *

# CV data initiation

In [35]:
# initiating data generators
feat_generator = FeatureGenerator()
test_generator = TestGenerator(train=True)

# initiating cv splitter
tscv = TimeSeriesSplit(test_size = 2, max_train_size=11)

# generating all of the data which we will iterate over during CV
target_df = test_generator.generate_target_for_month(month_nums=list(range(13, 34)))
features_df = test_generator.add_features_to_backbone(test_backbone=target_df, 
                                                      feat_generator=feat_generator)
short_df = features_df[:300000].copy()

# creating col lists for training
cols_di={
    'index': ['shop_id', 'item_id', 'date_block_num'],
    'target': ['target'],
    'feats': feat_generator.base_feat_cols + feat_generator.lag_cols + feat_generator.roll_cols
}

# Baseline

In [41]:
# initiating toy model
toy_model = ToyModel()

# iterating over CV folds
cv_res = run_cv(df=features_df, months_cv_split=tscv, model=toy_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22]
  Test months: [23 24]
  NRMSE:  1.0
  RMSE :  6.3


------------------------------
Fold 1:
  Train months: [15 16 17 18 19 20 21 22 23 24]
  Test months: [25 26]
  NRMSE:  1.0
  RMSE :  6.3


------------------------------
Fold 2:
  Train months: [17 18 19 20 21 22 23 24 25 26]
  Test months: [27 28]
  NRMSE:  1.0
  RMSE :  5.6


------------------------------
Fold 3:
  Train months: [19 20 21 22 23 24 25 26 27 28]
  Test months: [29 30]
  NRMSE:  1.0
  RMSE :  3.7


------------------------------
Fold 4:
  Train months: [21 22 23 24 25 26 27 28 29 30]
  Test months: [31 32]
  NRMSE:  1.0
  RMSE :  1.1e+01


------------------------------
RMSE mean: 6.6
NRMSE mean: 1.0


# Linear model

In [38]:
linear_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lin_model", ElasticNet(tol=1e-3)) #ElasticNet() Lasso(alpha=.005) , 
])

In [40]:
# initiating lin model
lin_model = linear_pipe

# iterating over CV folds
cv_res = run_cv(df=features_df, months_cv_split=tscv, model=lin_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22]
  Test months: [23 24]
  NRMSE:  1.0
  RMSE :  6.3


------------------------------
Fold 1:
  Train months: [15 16 17 18 19 20 21 22 23 24]
  Test months: [25 26]
  NRMSE:  1.0
  RMSE :  6.3


------------------------------
Fold 2:
  Train months: [17 18 19 20 21 22 23 24 25 26]
  Test months: [27 28]
  NRMSE:  1.0
  RMSE :  5.6


------------------------------
Fold 3:
  Train months: [19 20 21 22 23 24 25 26 27 28]
  Test months: [29 30]
  NRMSE:  1.0
  RMSE :  3.7


------------------------------
Fold 4:
  Train months: [21 22 23 24 25 26 27 28 29 30]
  Test months: [31 32]
  NRMSE:  1.0
  RMSE :  1.1e+01


------------------------------
RMSE mean: 6.5
NRMSE mean: 1.0


# Tree based

In [29]:
# tree_model = RandomForestRegressor(n_estimators=100)
tree_model = XGBRegressor()

cv_res = run_cv(df=features_df, months_cv_split=tscv, model=tree_model, cols_di=cols_di, verbose=1)

RMSE mean: 6.5
NRMSE mean: 1.0


In [31]:
cv_res

{'rmse': [6.353303703165361,
  6.3084489609241325,
  5.522449702002815,
  3.66291411755961,
  10.852254891927835],
 'nrmse': [1.0029327096592144,
  0.9956128262359158,
  0.9945574782650259,
  0.9909913851187914,
  1.0008508942502616],
 'train_months': [array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22]),
  array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
  array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]),
  array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
  array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])],
 'test_months': [array([23, 24]),
  array([25, 26]),
  array([27, 28]),
  array([29, 30]),
  array([31, 32])]}

# Recurrent NN